In [1]:
import sys
sys.path.append('/Users/tunadorable/local-repos/ng-video-lecture/venv/lib/python3.11/site-packages')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import time
import numpy as np

In [3]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [4]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 20
n_embd = 128
n_head = 8
n_layer = 8
dropout = 0.2

In [5]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars, vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 65


In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [8]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # sets model to eval mode
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # just resets to training mode
    return out

In [11]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [12]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        self.sa = nn.MultiheadAttention(embed_dim=n_embd, num_heads=n_head)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self._sa_forward(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

    def _sa_forward(self, x):
        # MultiheadAttention expects inputs in (t, b, d) format.
        x = x.transpose(0, 1)  # Convert (b, t, d) to (t, b, d)
        attn_output, _ = self.sa(x, x, x)
        return attn_output.transpose(0, 1)  # Convert back to (b, t, d)

In [13]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # learnable positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) 
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, verbose=None):
        b, t = idx.shape
        
        # idx and targets are both (b,t) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (b,t,d)
        pos_emb = self.position_embedding_table(torch.arange(t, device=device)) # (t,d)
        x0 = tok_emb + pos_emb # (b,t,d) + (t,d) = (b,t,d)
        if verbose: print("x0: ", x0)
            
        x = self.ln_f(self.blocks(x0)) # (b,t,d) -> (b,t,d)

        # Use the transposed embedding matrix for the output layer
        logits = x @ self.token_embedding_table.weight.t() # (b,t,d)@(d,v)=(b,t,v)

        p_max = torch.max(F.softmax(logits, dim=2), dim=2).values # (b,t)
        if verbose: print("p_max: ", p_max)

        # Convert p_max to numpy array for percentile calculation
        gamma = np.percentile(p_max.cpu().detach().numpy(), 25)
        if verbose: print("gamma: ", gamma)
            
        A_c = torch.zeros((b,t), device=device)
        A_c[p_max <= gamma] = 1 # (b,t)
        if verbose: print("A_c: ", A_c)

        C = x*A_c.unsqueeze(dim=2)
        if verbose: print("C: ", C)
            
        x0 = self.ln_f(x0 + C)
        if verbose: print("x0': ", x0)
        
        x = self.ln_f(self.blocks(x0)) # (b,t,d) -> (b,t,d)
        logits = x @ self.token_embedding_table.weight.t() # (b,t,d)@(d,v)=(b,t,v)
        
        if targets is None:
            loss = None
        else:
            b, t, v = logits.shape
            logits = logits.view(b*t, v)
            targets = targets.view(b*t)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (b, t) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (b, d)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (b, d)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (b, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (b, t+1)
        return idx

In [14]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')

1602.944 K parameters


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [16]:
start_time = time.time()
for iter in range(max_iters):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # train
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        current_time = time.time()
        elapsed_time = current_time - start_time
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time elapsed: {elapsed_time:.2f} seconds")

step 0: train loss 4.1109, val loss 4.1094, time elapsed: 2.48 seconds
step 500: train loss 2.4545, val loss 2.4488, time elapsed: 95.84 seconds
step 1000: train loss 2.2904, val loss 2.3090, time elapsed: 181.94 seconds
step 1500: train loss 2.2009, val loss 2.2186, time elapsed: 260.46 seconds
step 2000: train loss 0.0709, val loss 0.0692, time elapsed: 343.32 seconds
step 2500: train loss 0.0457, val loss 0.0480, time elapsed: 425.89 seconds
step 3000: train loss 0.0435, val loss 0.0448, time elapsed: 504.76 seconds
step 3500: train loss 0.0422, val loss 0.0426, time elapsed: 582.57 seconds
step 4000: train loss 0.0440, val loss 0.0452, time elapsed: 660.17 seconds
step 4500: train loss 0.0396, val loss 0.0418, time elapsed: 746.52 seconds
step 5000: train loss 0.0408, val loss 0.0393, time elapsed: 828.87 seconds
step 5500: train loss 0.0350, val loss 0.0350, time elapsed: 915.69 seconds
step 6000: train loss 0.0385, val loss 0.0399, time elapsed: 999.08 seconds
step 6500: train lo

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 20
n_embd = 128
n_head = 8
n_layer = 8
dropout = 0.2

In [18]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=64)[0].tolist()))


veersssrsssessssssssssssswseay ssssssssssssensssssssssssssssssme


In [210]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

1.446 K parameters


In [23]:
# sample a batch of data
xb, yb = get_batch('train')
    
# train
logits, loss = model(xb, yb, verbose=True) 
optimizer.zero_grad(set_to_none=True)
loss.backward()
optimizer.step()

x0:  tensor([[[-0.0136,  0.0092, -0.0118,  ..., -0.0553,  0.0360, -0.0220],
         [-0.0046, -0.0236,  0.0215,  ..., -0.0279,  0.0226,  0.0229],
         [-0.0147,  0.0345,  0.0186,  ..., -0.0043,  0.0852,  0.0075],
         ...,
         [-0.0376,  0.0483, -0.0247,  ..., -0.0535,  0.0144, -0.0107],
         [-0.0059, -0.0480,  0.0746,  ..., -0.0128,  0.0289,  0.0463],
         [-0.0364, -0.0154,  0.0305,  ..., -0.0136, -0.0470, -0.0195]],

        [[-0.0440, -0.0339,  0.0090,  ..., -0.0284,  0.0501,  0.0362],
         [-0.0172,  0.0026, -0.0138,  ..., -0.0118, -0.0127,  0.0078],
         [ 0.0102,  0.0030, -0.0060,  ...,  0.0113,  0.0886,  0.0357],
         ...,
         [-0.0526, -0.0334,  0.0317,  ..., -0.0493,  0.0061,  0.0172],
         [ 0.0200, -0.0147,  0.0744,  ...,  0.0025,  0.0149,  0.0463],
         [-0.0355, -0.0025,  0.0296,  ...,  0.0136,  0.0125,  0.0214]],

        [[-0.0378, -0.0729,  0.0477,  ..., -0.0610,  0.0211,  0.0089],
         [-0.0017,  0.0251, -0.0265,  ..

C:  tensor([[[-0.4329, -1.1162,  0.9776,  ..., -0.3292,  0.9623,  0.5216],
         [-0.4012, -1.1405,  0.7272,  ..., -0.0615,  0.9501,  0.6382],
         [-0.5243, -1.2002,  1.0042,  ...,  0.0748,  1.0177,  0.5984],
         ...,
         [-0.5150, -1.0770,  1.0807,  ..., -0.2228,  1.0351,  0.5149],
         [-0.2192, -0.9870,  1.3905,  ..., -0.0297,  1.0691,  0.5958],
         [-0.4690, -1.3092,  1.5090,  ..., -0.0268,  0.8823,  0.5285]],

        [[-0.8486, -1.2228,  0.6737,  ..., -1.2504,  0.4007,  0.2683],
         [-0.7499, -1.4917,  0.8430,  ..., -0.9296,  0.3643,  0.7370],
         [-0.7285, -1.6327,  0.5873,  ..., -1.2083,  0.4429,  0.7145],
         ...,
         [-0.7679, -1.3772,  0.9341,  ..., -0.8925,  0.0672,  0.5105],
         [-0.6459, -1.5165,  0.7143,  ..., -0.8643,  0.6059,  0.4397],
         [-0.6970, -1.4965,  1.2389,  ..., -0.4259,  0.5794,  0.7010]],

        [[-0.5839, -1.6621,  1.0294,  ..., -1.0141,  0.4355,  0.5001],
         [-0.5389, -1.2075,  0.8798,  ...

A_c:  tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1.

p_max:  tensor([[0.1180, 0.1209, 0.1110, 0.1195, 0.0985, 0.1201, 0.1243, 0.1176, 0.1216,
         0.1170, 0.1113, 0.1237, 0.1214, 0.1164, 0.1147, 0.1168, 0.1213, 0.1005,
         0.1196, 0.1223, 0.1180, 0.0905, 0.0984, 0.1157, 0.1231, 0.1174, 0.1111,
         0.1176, 0.1166, 0.1145, 0.1228, 0.1103],
        [0.1182, 0.1172, 0.1218, 0.1219, 0.1183, 0.1158, 0.1208, 0.1070, 0.1206,
         0.1164, 0.1182, 0.1182, 0.1190, 0.1217, 0.1192, 0.1211, 0.1161, 0.1212,
         0.1211, 0.1211, 0.1180, 0.1189, 0.1210, 0.1191, 0.1167, 0.1207, 0.1172,
         0.1163, 0.1188, 0.1241, 0.1187, 0.1145],
        [0.1171, 0.1165, 0.1225, 0.0594, 0.1179, 0.1205, 0.1141, 0.0617, 0.0693,
         0.0593, 0.1170, 0.1187, 0.1222, 0.0774, 0.1241, 0.1179, 0.1176, 0.1160,
         0.1202, 0.1194, 0.1170, 0.1186, 0.1189, 0.1159, 0.1169, 0.1224, 0.1146,
         0.1183, 0.1203, 0.1225, 0.1104, 0.1151],
        [0.1296, 0.0863, 0.1273, 0.1252, 0.1251, 0.1301, 0.1296, 0.1285, 0.1299,
         0.1262, 0.1227, 0.1262,

A_c:  tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1.

p_max:  tensor([[0.1210, 0.1225, 0.1154, 0.1221, 0.1054, 0.1219, 0.1243, 0.1203, 0.1235,
         0.1196, 0.1144, 0.1237, 0.1230, 0.1197, 0.1178, 0.1192, 0.1230, 0.1068,
         0.1207, 0.1229, 0.1210, 0.0973, 0.1051, 0.1188, 0.1239, 0.1209, 0.1151,
         0.1213, 0.1197, 0.1183, 0.1228, 0.1151],
        [0.1225, 0.1217, 0.1247, 0.1243, 0.1219, 0.1223, 0.1220, 0.1157, 0.1238,
         0.1204, 0.1221, 0.1221, 0.1223, 0.1240, 0.1224, 0.1243, 0.1201, 0.1236,
         0.1236, 0.1239, 0.1227, 0.1207, 0.1237, 0.1223, 0.1219, 0.1235, 0.1211,
         0.1198, 0.1230, 0.1256, 0.1209, 0.1198],
        [0.1208, 0.1209, 0.1244, 0.0633, 0.1214, 0.1241, 0.1199, 0.0664, 0.0705,
         0.0617, 0.1212, 0.1214, 0.1245, 0.0903, 0.1249, 0.1213, 0.1214, 0.1215,
         0.1229, 0.1233, 0.1203, 0.1221, 0.1214, 0.1199, 0.1200, 0.1249, 0.1191,
         0.1217, 0.1227, 0.1241, 0.1178, 0.1196],
        [0.1309, 0.0973, 0.1301, 0.1275, 0.1285, 0.1309, 0.1302, 0.1310, 0.1313,
         0.1286, 0.1265, 0.1291,

A_c:  tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1.

p_max:  tensor([[0.1219, 0.1225, 0.1171, 0.1225, 0.1096, 0.1230, 0.1233, 0.1217, 0.1236,
         0.1201, 0.1165, 0.1225, 0.1241, 0.1208, 0.1192, 0.1204, 0.1231, 0.1117,
         0.1217, 0.1248, 0.1211, 0.1018, 0.1099, 0.1196, 0.1239, 0.1216, 0.1174,
         0.1211, 0.1207, 0.1203, 0.1235, 0.1162],
        [0.1237, 0.1233, 0.1251, 0.1252, 0.1233, 0.1239, 0.1237, 0.1206, 0.1248,
         0.1228, 0.1230, 0.1241, 0.1237, 0.1243, 0.1239, 0.1253, 0.1223, 0.1244,
         0.1247, 0.1243, 0.1228, 0.1220, 0.1246, 0.1240, 0.1234, 0.1243, 0.1231,
         0.1210, 0.1248, 0.1262, 0.1234, 0.1221],
        [0.1220, 0.1241, 0.1247, 0.0652, 0.1226, 0.1248, 0.1228, 0.0681, 0.0693,
         0.0657, 0.1234, 0.1227, 0.1256, 0.0998, 0.1247, 0.1223, 0.1228, 0.1228,
         0.1246, 0.1242, 0.1229, 0.1231, 0.1229, 0.1224, 0.1230, 0.1253, 0.1224,
         0.1230, 0.1236, 0.1246, 0.1206, 0.1219],
        [0.1312, 0.1051, 0.1297, 0.1280, 0.1297, 0.1302, 0.1308, 0.1307, 0.1304,
         0.1291, 0.1280, 0.1302,

C:  tensor([[[-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         ...,
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000]],

        [[-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         ...,
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000]],

        [[-0.0000, -0.0000,  0.0000,  ..., -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000,  0.0000,  ...